# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [3]:
from crewai import Agent, Crew, Task

In [4]:
import os
from utils import get_serper_api_key

# Serper is an API that allows you to search Google.
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [5]:
# Initiate our LLM using Google AI.
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAI

load_dotenv()
api_key = os.environ['GOOGLE_API_KEY']

llm = GoogleGenerativeAI(model="models/gemini-pro", google_api_key=api_key)

## crewAI Tools

In [6]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [7]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue " "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=llm,
)

In [8]:
# Agent 2: Logistics Manager
logistics_manager = Agent(
    role="Logistics Manager",
    goal=("Manage all logistics for the event " "including catering and equipmen"),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=llm,
)

In [9]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and " "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=llm,
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [10]:
# Pydantic is a data validation and settings management library for Python, 
#   which provides a way to define data models with type annotations and validation rules.

from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    """
        This is a very interesting use case on how you can use agents to transfer the
            fuzzy inputs into actual strongly-typed outputs.  
    """
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [11]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [12]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [13]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [14]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

### ⚠️ NOTE 
The following code almost will throw an exception due to sending many requests to Google API overcoming the limit. 
For the problem details, see this [conversation](https://groups.google.com/g/adwords-api/c/gcKvh1C3GX4?pli=1)

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

- Display the generated `venue_details.json` file.

In [18]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'San Francisco, CA',
 'booking_status': 'Available',
 'capacity': 1000,
 'name': 'Fairmont San Francisco'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [20]:
from IPython.display import Markdown
Markdown("marketing_report.md")

**Marketing Activities and Attendee Engagement Plan**

**Marketing Activities:**

* **Content Marketing:**
    * Create and promote blog posts, articles, infographics, and videos focusing on tech innovation trends and industry insights.
* **Social Media Marketing:**
    * Establish a strong social media presence on relevant platforms (e.g., Twitter, LinkedIn, Facebook).
    * Engage in targeted social media campaigns promoting the conference.
* **Email Marketing:**
    * Build an email list of potential attendees.
    * Send out regular email updates with conference news, speaker announcements, and early bird registration offers.
* **Paid Advertising:**
    * Consider using Google AdWords and social media advertising to reach a wider audience.
* **Public Relations:**
    * Reach out to industry publications and media outlets to generate coverage for the conference.

**Attendee Engagement:**

* **Dedicated Conference Website:**
    * Create a comprehensive website providing all necessary information about the conference, including schedule, speakers, and registration details.
* **Online Community:**
    * Establish an online community platform (e.g., a forum or discussion board) to foster interaction among potential attendees and industry professionals.
* **Exclusive Content and Resources:**
    * Offer exclusive content, such as whitepapers and webinars, to engaged attendees.
* **Social Media Engagement:**
    * Encourage attendees to share their experiences and connect with other participants using a dedicated conference hashtag.
* **Networking Opportunities:**
    * Plan networking events and ice breakers to facilitate connections and knowledge exchange.